# Generació del Mapa Interactiu de Trànsit Aeri i Qualitat de l'Aire

Aquest notebook crea un mapa interactiu que integra les dades de trànsit aeri (ADS-B) i qualitat de l'aire (ICA) utilitzant Folium.

### Objectiu
- Representar geogràficament la densitat de vols sobre el territori espanyol.
- Visualitzar l'estat de les estacions de control de la qualitat de l'aire.
- Integrar components interactius per explorar la relació temporal i espacial.
- Crear una eina visual per a la presentació dels resultats del TFG.

### Prerequisits
Aquest notebook requereix que s'hagi executat prèviament el **Notebook 02: Neteja i Preprocessament**, que genera els fitxers:
- `adsb_clean.csv` (amb grid espacial i columna `hora`)
- `ica_clean.csv` (amb colors ICA i categories)

### Abast
- Càrrega de dades preprocessades
- Agregació de dades per a visualització
- Configuració de capes dinàmiques (HeatMap, punts, animacions)
- Generació del mapa HTML final

In [1]:
# Importar les biblioteques necessàries
import pandas as pd
import folium
from folium import plugins
from pathlib import Path
import json
import warnings
warnings.filterwarnings('ignore')

/tmp/ipykernel_72/590285140.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


## 1. Carregar dades preprocessades

In [2]:
# Llegir els fitxers CSV preprocessats
adsb_path = Path("../data/processed/adsb/adsb_clean.csv")
df_adsb = pd.read_csv(adsb_path)

ica_path = Path("../data/processed/ica/ica_clean.csv")
df_ica = pd.read_csv(ica_path)

# Convertir columnes temporals a datetime
if 'timestamp' in df_adsb.columns:
    df_adsb['timestamp'] = pd.to_datetime(df_adsb['timestamp'])
df_adsb['hora'] = pd.to_datetime(df_adsb['hora'])
df_ica['hora'] = pd.to_datetime(df_ica['hora'])

print(f"Fitxer ADS-B: {len(df_adsb)} registres, {len(df_adsb.columns)} columnes")
print(f"Fitxer ICA: {len(df_ica)} registres, {len(df_ica.columns)} columnes")

Fitxer ADS-B: 75989 registres, 19 columnes
Fitxer ICA: 12625 registres, 11 columnes


## 2. Preparar dades per a visualització del mapa

Abans de generar el mapa, es realitza una agregació i un mostreig de les dades per optimitzar el rendiment de la visualització i garantir que la informació sigui llegible:

- Estacions ICA (agregació per estació): Es consoliden les dades de qualitat de l'aire per cada estació física. Es calcula la mitjana de l'índex, el contaminant principal més freqüent i el percentatge de presència de cada substància detectada.

- Top zones de trànsit aeri: S'agrupen les deteccions ADS-B per cel·les geogràfiques per identificar els "punts calents" de l'espai aeri. Es realitza un filtratge manual de coordenades per excloure zones fora de l'estudi (com el sud de França) i es seleccionen les 20 àrees amb més activitat.

- Mostreig per a HeatMap: Es seleccionen fins a 15.000 punts aleatoris del conjunt ADS-B. Aquesta tècnica de sampling permet generar un mapa de calor representatiu de tot el territori sense alentir el navegador de l'usuari.

- Mostreig per a l'Animació Temporal: Es filtren 3.000 registres de vols i s'ordenen cronològicament. Aquesta selecció és essencial perquè el control de temps (slider) funcioni de manera fluida i mostri el moviment dels avions d'una forma coherent.

In [3]:
# Estacions ICA - Agregació per estació
estacions = df_ica.groupby(['cod_estacion', 'nombre']).agg({
    'lat': 'first',
    'lon': 'first',
    'indice': 'mean', # mitjana
    'tipo': 'first',
    'debido_a': lambda x: list(x.unique()), # únics per evitar repeticions llargues
    'QualitatText': lambda x: x.mode()[0] if not x.empty else 'Sense dades',
    'ColorICA': 'first'
}).reset_index()

# Funció per calcular percentatges de contaminants
def calcular_percentatges(lista):
    if len(lista) == 0:
        return {}
    counts = pd.Series(lista).value_counts()
    total = len(lista)
    return {cont: (count/total)*100 for cont, count in counts.items()}

estacions['contaminants_pct'] = estacions['debido_a'].apply(calcular_percentatges)
estacions['contaminant_principal'] = estacions['debido_a'].apply(
    lambda x: pd.Series(x).mode()[0] if len(x) > 0 else 'Desconegut'
)

print(f"Estacions ICA: {len(estacions)}")
print(f"\nDistribució qualitat aire per estació:")
print(estacions['QualitatText'].value_counts())

Estacions ICA: 505

Distribució qualitat aire per estació:
QualitatText
Bona         502
Sin datos      3
Name: count, dtype: int64


In [4]:
# Top zones de trànsit aeri (agregació per cell_id)
zones_traffic = df_adsb.groupby('cell_id').agg({
    'hex': 'count',
    'lat_centre': 'first',
    'lon_centre': 'first'
}).reset_index()
zones_traffic.columns = ['cell_id', 'num_deteccions', 'lat_centre', 'lon_centre']

# Excloure zones de França
zones_excloure = ['17_22', '17_21', '16_21', '17_23'] # ID d'estacions de zones específiques de França
zones_traffic = zones_traffic[~zones_traffic['cell_id'].isin(zones_excloure)]

# Filtratge addicional per coordenades d'Espanya
zones_traffic_spain = zones_traffic[
    (zones_traffic['lat_centre'].between(36.0, 43.8)) &
    (zones_traffic['lon_centre'].between(-9.5, 4.5))
]

# Top 20 zones
top_zones = zones_traffic_spain.nlargest(20, 'num_deteccions')

print(f"Zona més transitada: {top_zones.iloc[0]['num_deteccions']:,} vols")

Zona més transitada: 581 vols


In [5]:
# Mostreig d'avions per HeatMap
MAX_POINTS_HEAT = 15000
df_adsb_heat = df_adsb.sample(n=min(MAX_POINTS_HEAT, len(df_adsb)), random_state=42)

# Mostreig d'avions per animació
MAX_POINTS_ANIM = 3000
df_adsb_anim = df_adsb.sample(n=min(MAX_POINTS_ANIM, len(df_adsb)), random_state=42)
df_adsb_anim = df_adsb_anim.sort_values('timestamp')  # Important: ordenar per temps!

print(f"Avions per HeatMap: {len(df_adsb_heat):,}")
print(f"Avions per animació: {len(df_adsb_anim):,}")

Avions per HeatMap: 15,000
Avions per animació: 3,000


## 3. Crear mapa base i afegir capes al mapa

El mapa resultant es divideix en quatre capes principals que l'usuari pot activar o desactivar:

- Mapa de Calor (HeatMap): Visualitza la densitat global del trànsit aeri, ressaltant les zones de major activitat mitjançant un gradient de colors.

- Estacions ICA: Representació de les estacions de mesura amb pop-ups detallats que inclouen el desglossament de contaminants i la classificació de la qualitat de l'aire.

- Top 20 Zones de Trànsit: Identificació clara de les àrees amb més deteccions mitjançant indicadors circulars.

- Animació Temporal (Timestamped GeoJSON): Una capa dinàmica que permet observar el moviment dels avions en franges de 5 minuts mitjançant un control de temps (slider).

In [6]:
# Crear mapa centrat a Espanya
mapa = folium.Map(
    location=[40.4, -3.7],  # Madrid
    zoom_start=6,
    tiles='OpenStreetMap',
    control_scale=True
)

In [7]:
# CAPA 1: HeatMap de densitat de vols
heat_data = [[row['lat'], row['lon']] for _, row in df_adsb_heat.iterrows()]

plugins.HeatMap(
    heat_data,
    min_opacity=0.2,
    max_zoom=13,
    radius=15,
    blur=20,
    gradient={
        0.4: 'blue',
        0.5: 'cyan',
        0.6: 'lime',
        0.7: 'yellow',
        0.8: 'orange',
        1.0: 'red'
    },
    name='Mapa de Calor Trànsit'
).add_to(mapa)

# CAPA 2: Estacions ICA
fg_estacions = folium.FeatureGroup(name='Estacions ICA', show=True)

for _, est in estacions.iterrows():
    # Crear HTML per contaminants
    contaminants_html = ""
    if est['contaminants_pct']:
        contaminants_html = "<br><br><b>Contaminants detectats:</b><br>"
        for cont, pct in sorted(est['contaminants_pct'].items(), key=lambda x: x[1], reverse=True):
            contaminants_html += f"&nbsp;&nbsp;• {cont}: {pct:.1f}%<br>"

    # Crear punt de color visual
    color_dot = f'<span style="display: inline-block; width: 12px; height: 12px; ' \
                f'background-color: {est["ColorICA"]}; border-radius: 50%; ' \
                f'border: 2px solid black; margin-right: 5px;"></span>'

    # Crear popup amb informació completa
    popup_html = f"""
    <div style="font-family: Arial; font-size: 13px; min-width: 250px;">
        <h4 style="margin: 0 0 10px 0; color: #2c3e50; border-bottom: 2px solid #3498db; padding-bottom: 5px;">
            {est['nombre']}
        </h4>
        <b>ICA Mitjà:</b> {est['indice']:.1f}<br>
        <b>Qualitat:</b> {color_dot}<b style="color: {est['ColorICA']};">{est['QualitatText']}</b><br>
        <b>Tipus:</b> {est['tipo']}<br>
        <b>Principal:</b> {est['contaminant_principal']}<br>
        <b>Codi:</b> {est['cod_estacion']}
        {contaminants_html}
    </div>
    """

    # Afegir marcador amb color
    marker = folium.CircleMarker(
        location=[est['lat'], est['lon']],
        radius=8,
        popup=folium.Popup(popup_html, max_width=350),
        tooltip=f"{est['nombre']} (ICA: {est['indice']:.1f})",
        color='black',
        fillColor=est['ColorICA'],
        fillOpacity=0.8,
        weight=2
    )
    marker.options['className'] = f'estacio-{est["cod_estacion"]}'
    marker.add_to(fg_estacions)

mapa.add_child(fg_estacions)

# CAPA 3: Top zones de trànsit
fg_zones = folium.FeatureGroup(name='Top 20 Zones Trànsit', show=True)

for _, zona in top_zones.iterrows():
    popup_html = f"""
    <div style="font-family: Arial; font-size: 13px;">
        <h4 style="margin: 0 0 10px 0; color: #2c3e50; border-bottom: 2px solid #e74c3c; padding-bottom: 5px;">
            Zona de Trànsit Intens
        </h4>
        <b>ID:</b> {zona['cell_id']}<br>
        <b>Deteccions:</b> {int(zona['num_deteccions']):,}<br>
        <b>Coordenades:</b> {zona['lat_centre']:.2f}°N, {zona['lon_centre']:.2f}°E
    </div>
    """

    folium.Circle(
        location=[zona['lat_centre'], zona['lon_centre']],
        radius=25000,  # 25km de radi
        popup=folium.Popup(popup_html, max_width=250),
        tooltip=f"Zona: {int(zona['num_deteccions']):,} deteccions",
        color='navy',
        fillColor='blue',
        fillOpacity=0.15,
        weight=2,
        dashArray='5, 5'
    ).add_to(fg_zones)

mapa.add_child(fg_zones)

# CAPA 4: Animació temporal de vols

# Preparar dades per TimestampedGeoJson
features = []
for _, row in df_adsb_anim.iterrows():
    # Gestionar nom del vol
    flight_name = row.get('flight', 'Desconegut')
    if pd.isna(flight_name) or flight_name == '':
        flight_name = f"Avió {row.get('hex', 'N/A')}"

    feature = {
        'type': 'Feature',
        'geometry': {
            'type': 'Point',
            'coordinates': [row['lon'], row['lat']]
        },
        'properties': {
            'time': row['timestamp'].isoformat(),
            'popup': f"<b>{flight_name}</b><br>Alt: {row.get('alt_baro', 'N/A')} ft<br>Vel: {row.get('gs', 'N/A')} kt<br>Hora: {row['timestamp'].strftime('%H:%M')}",
            'icon': 'circle',
            'iconstyle': {
                'fillColor': '#ff6b6b',
                'fillOpacity': 0.8,
                'stroke': 'true',
                'color': 'darkred',
                'weight': 2,
                'radius': 6
            }
        }
    }
    features.append(feature)

# Afegir animació temporal
plugins.TimestampedGeoJson(
    {'type': 'FeatureCollection', 'features': features},
    period='PT5M',           # Interval de 5 minuts
    add_last_point=False,    # False perquè els avions desapareguin!
    auto_play=False,
    loop=False,
    max_speed=10,
    loop_button=True,
    date_options='HH:mm',
    time_slider_drag_update=True,
    duration='PT5M'          # Durada de visualització de cada punt (5 minuts)
).add_to(mapa)

print(f"Animació temporal creada amb {len(features)} avions")

Animació temporal creada amb 3000 avions


## 4. Afegir controls i interfície

S'ha implementat una interfície personalitzada mitjançant la injecció de codi HTML/CSS/JavaScript per millorar l'experiència d'usuari:

- Buscador d'estacions: Un panell de cerca amb autocompletat que permet localitzar estacions específiques pel seu nom.

- Targeta informativa dinàmica: En seleccionar una estació o un avió, es mostra un panell detallat que identifica automàticament la zona geogràfica i les estadístiques de contaminació/avió.

- Llegenda interactiva i punt d'informació: Guia visual per a la interpretació dels colors del mapa i els nivells d'ICA així com un panell amb informació de com funciona el mapa.

In [8]:
# Control de capes
folium.LayerControl(position='topright', collapsed=False).add_to(mapa)

In [9]:
# HTML/CSS/JavaScript per controls
estacions_json = estacions[['cod_estacion', 'nombre', 'lat', 'lon']].to_dict('records')

custom_html = f"""
<style>
.control-btn {{
    position: absolute;
    z-index: 1000;
    background: white;
    padding: 10px 15px;
    border-radius: 8px;
    box-shadow: 0 2px 8px rgba(0,0,0,0.3);
    cursor: pointer;
    font-weight: bold;
    font-size: 13px;
    transition: all 0.3s;
}}
.control-btn:hover {{
    background: #f0f0f0;
    transform: scale(1.05);
}}
.control-card {{
    position: absolute;
    z-index: 1000;
    background: white;
    padding: 15px;
    border-radius: 8px;
    box-shadow: 0 4px 12px rgba(0,0,0,0.3);
    display: none;
}}
</style>

<script>
const estacions = {json.dumps(estacions_json)};
let currentFilter = null;

function toggleInfo() {{
    const card = document.getElementById('info-card');
    card.style.display = card.style.display === 'none' ? 'block' : 'none';
}}

function toggleSearch() {{
    const card = document.getElementById('search-card');
    card.style.display = card.style.display === 'none' ? 'block' : 'none';
}}

function toggleLegend() {{
    const card = document.getElementById('legend-card');
    card.style.display = card.style.display === 'none' ? 'block' : 'none';
}}

function filterStation(codEstacio) {{
    currentFilter = codEstacio;
    document.querySelectorAll('[class*="estacio-"]').forEach(el => {{
        el.style.display = 'none';
    }});
    const selected = document.querySelector('.estacio-' + codEstacio);
    if (selected) selected.style.display = 'block';
    document.getElementById('filter-status').style.display = 'block';

    const estacio = estacions.find(e => e.cod_estacion === codEstacio);
    if (estacio) {{
        const map = document.querySelector('.folium-map');
        if (map && map._leaflet_map) {{
            map._leaflet_map.setView([estacio.lat, estacio.lon], 12);
        }}
    }}
}}

function resetFilter() {{
    currentFilter = null;
    document.querySelectorAll('[class*="estacio-"]').forEach(el => {{
        el.style.display = 'block';
    }});
    document.getElementById('filter-status').style.display = 'none';
    document.getElementById('search-input').value = '';
    document.getElementById('search-results').innerHTML = '';
}}

document.addEventListener('DOMContentLoaded', function() {{
    const searchInput = document.getElementById('search-input');
    if (searchInput) {{
        searchInput.addEventListener('input', function() {{
            const query = this.value.toLowerCase().trim();
            const results = document.getElementById('search-results');
            results.innerHTML = '';
            if (query.length < 2) return;
            const matches = estacions.filter(e => e.nombre.toLowerCase().includes(query)).slice(0, 10);
            if (matches.length === 0) {{
                results.innerHTML = '<div style="padding: 5px; color: #999;">No s\\'han trobat estacions</div>';
                return;
            }}
            matches.forEach(est => {{
                const div = document.createElement('div');
                div.style.cssText = 'padding: 8px; border-bottom: 1px solid #eee; cursor: pointer; font-size: 12px;';
                div.innerHTML = '<b>' + est.nombre + '</b><br><small>Codi: ' + est.cod_estacion + '</small>';
                div.onmouseenter = () => div.style.backgroundColor = '#f0f0f0';
                div.onmouseleave = () => div.style.backgroundColor = 'white';
                div.onclick = () => filterStation(est.cod_estacion);
                results.appendChild(div);
            }});
        }});
    }}
}});
</script>

<!-- Botó Info -->
<div class="control-btn" style="top: 10px; left: 70px;" onclick="toggleInfo()">
    ℹ️ Informació
</div>

<div class="control-card" id="info-card" style="top: 60px; left: 70px; width: 450px; font-size: 14px;">
    <h3 style="margin: 0 0 8px 0; color: #2c3e50;">Mapa de Trànsit Aeri i Qualitat de l'Aire d'Espanya</h3>
    <hr style="margin: 5px 0; border-top: 2px solid #3498db;">
    <small style="color: #34495e;">
        <b>Ubicació:</b> Espanya | <b>Període:</b> 24 hores<br><br>
        <b>Controls:</b><br>
        • <b>Slider temporal</b> per veure el trànsit d'avions (inferior esquerra)<br>
        • <b>Llegenda</b> per interpretar el mapa (inferior dreta)<br>
        • <b>Control de capes</b> per activar o desactivar diferents capes (dalt dreta)<br>
        • <b>Buscador</b> per buscar i filtrar estacions en concret (dalt centre)<br>
        • <b>Clica</b> elements del mapa per veure amb més detalls
    </small>
</div>

<!-- Botó Buscador -->
<div class="control-btn" style="top: 10px; left: 50%; transform: translateX(-50%);" onclick="toggleSearch()">
    🔍 Buscar Estació ICA
</div>

<div class="control-card" id="search-card" style="top: 60px; left: 50%; transform: translateX(-50%); width: 350px;">
    <div style="display: flex; justify-content: space-between; align-items: center; margin-bottom: 10px;">
        <div style="font-size: 14px; font-weight: bold; color: #2c3e50;">Buscar Estació</div>
        <button onclick="resetFilter()" style="padding: 5px 10px; border: 1px solid #2c3e50; background: white;
                border-radius: 4px; cursor: pointer; font-size: 12px; color: #2c3e50;">
            🔄 Reiniciar filtre
        </button>
    </div>
    <input type="text" id="search-input" placeholder="Escriu el nom de l'estació..."
           style="width: 100%; padding: 8px; border: 1px solid #ccc; border-radius: 4px; font-size: 13px; box-sizing: border-box;">
    <div id="search-results" style="max-height: 250px; overflow-y: auto; margin-top: 8px;"></div>
    <div id="filter-status" style="margin-top: 10px; padding: 8px; background: #e8f5e9;
         border-radius: 4px; font-size: 12px; color: #2e7d32; display: none;">
        ✓ Filtre actiu
    </div>
</div>

<!-- Botó Llegenda -->
<div class="control-btn" style="bottom: 50px; right: 50px;" onclick="toggleLegend()">
    📊 Llegenda
</div>

<div class="control-card" id="legend-card" style="bottom: 100px; right: 50px; width: 350px; font-size: 13px;">
    <h4 style="margin: 0 0 10px 0; text-align: center; color: #2c3e50;">📊 Llegenda</h4>
    <hr style="margin: 5px 0; border-top: 2px solid #34495e;">

    <b style="color: #2c3e50;">Qualitat de l'Aire:</b><br>
    <span style="display: inline-block; width: 12px; height: 12px; background-color: rgb(56, 162, 206);
          border-radius: 50%; border: 1px solid black; margin: 2px 5px;"></span> Bona<br>
    <span style="display: inline-block; width: 12px; height: 12px; background-color: rgb(50, 161, 94);
          border-radius: 50%; border: 1px solid black; margin: 2px 5px;"></span> Raonablement bona<br>
    <span style="display: inline-block; width: 12px; height: 12px; background-color: rgb(241, 229, 73);
          border-radius: 50%; border: 1px solid black; margin: 2px 5px;"></span> Regular<br>
    <span style="display: inline-block; width: 12px; height: 12px; background-color: rgb(200, 52, 65);
          border-radius: 50%; border: 1px solid black; margin: 2px 5px;"></span> Desfavorable<br>
    <span style="display: inline-block; width: 12px; height: 12px; background-color: rgb(110, 22, 29);
          border-radius: 50%; border: 1px solid black; margin: 2px 5px;"></span> Molt desfavorable<br>
    <span style="display: inline-block; width: 12px; height: 12px; background-color: rgb(162, 91, 164);
          border-radius: 50%; border: 1px solid black; margin: 2px 5px;"></span> Extremadament desfavorable<br>
    <span style="display: inline-block; width: 12px; height: 12px; background-color: rgb(85, 89, 93);
          border-radius: 50%; border: 1px solid black; margin: 2px 5px;"></span> Sense dades<br>
          <small>(Categories oficials basades en el contaminant pitjor - Resolució MITECO 2020, vigent el 2026)</small>
    <br>

    <br>
    <b style="color: #2c3e50;">Avions:</b><br>
    <span style="display: inline-block; width: 10px; height: 10px; background-color: #ff6b6b;
          border-radius: 50%; border: 1px solid darkred; margin: 2px 5px;"></span> Cada punt representa un avió<br>
    <br>

    <b style="color: #2c3e50;">Mapa de Calor:</b><br>
    Densitat de trànsit aeri (de menys a més)
    <div style="background: linear-gradient(to right, blue, cyan, lime, yellow, orange, red);
         height: 15px; border-radius: 3px; margin: 5px 0;"></div>
</div>
"""

mapa.get_root().html.add_child(folium.Element(custom_html))

## 5. Guardar mapa

In [10]:
# Guardar mapa HTML
output_path = Path("../outputs/maps/interactive_map.html")
output_path.parent.mkdir(parents=True, exist_ok=True)

mapa.save(str(output_path))

### Conclusió
Aquest notebook culmina amb la generació d'un fitxer HTML autònom que integra tota la informació del projecte. Aquesta eina permet una anàlisi exploratòria visual i interactiva, facilitant la comprensió de com interaccionen el trànsit aeri i la qualitat de l'aire en diferents punts de la geografia espanyola.